# Setup

In [ ]:
%pip install astropy
%pip install git+https://github.com/pvigier/perlin-numpy
%pip install tensorflow_addons
%pip install wandb

In [ ]:
!git clone https://github.com/tomasmuzas/phd

In [1]:
from phd.Augmentations.common_code.augmentation_functions import *
from phd.Augmentations.common_code.get_dataset import *
from phd.Augmentations.common_code.models import *
from phd.Augmentations.common_code.training_loop import *
from phd.Augmentations.common_code.utils import *
import tensorflow as tf

tf.get_logger().setLevel('ERROR')

In [2]:
from google.colab import auth

auth.authenticate_user()
reset_tpu(training_config)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Running on GPU.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [ ]:
!apt-get update -y
!curl -LJO "https://github.com/GoogleCloudPlatform/gcsfuse/releases/download/v1.2.0/gcsfuse_1.2.0_amd64.deb"
!apt-get -y install fuse
!apt-get clean
!dpkg -i "gcsfuse_1.2.0_amd64.deb"
!mkdir gcs
!gcsfuse --implicit-dirs tomasmuzas-phd-datasets gcs

In [3]:
import wandb

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: muzas-tomas (tomasmuzas-phd). Use `wandb login --relogin` to force relogin


True

# Training

In [ ]:
tf.get_logger().setLevel('ERROR')


training_config = {
"ENABLE_DETERMINISM": False,
"SEED": 17,
"AUTO": tf.data.AUTOTUNE,
"TRAIN_BATCH_SIZE": 1024,
"TEST_BATCH_SIZE": 256,
"NUMBER_OF_EPOCHS": 500,
"EARLY_STOPPING_TOLERANCE": 10,
"LEARNING_RATE": 1e-4,
"IMAGE_SIZE": 128,
"SHUFFLE_BUFFER": 300000,
"TPU": True,
"AUGMENTATIONS_ROTATE": True,
"AUGMENTATIONS_FLIP_HORIZONTALLY": True,
"AUGMENTATIONS_FLIP_VERTICALLY": True,
"AUGMENTATIONS_ZOOM": True,
"AUGMENTATIONS_RANDOM_NOISE": False,
"AUGMENTATIONS_CENTER_NOISE": False,
"AUGMENTATIONS_OUTSIDE_NOISE": False,
"AUGMENTATIONS_PERLIN_CENTER_NOISE": False,
"USE_ADABELIEF_OPTIMIZER": False,
"GALAXY_DATASET": "SDSS",
"TRAIN_DATASET_SIZE": 655360,
"TEST_DATASET_SIZE": 163809,
"STRETCH": False,
"STEPS_PER_EXECUTION": 1,
"FOLDS": 5,
"NUMBER_OF_CLASSES": 5
}
training_config["WANDB_PROJECT_NAME"] = f"Spiral Certainty Bins"
training_config["EXPERIMENT_DESCRIPTION"] = f"spiral_certainty_bins_main_augmentations_{training_config['FOLDS']}_folds"
training_config["LOCAL_GCP_PATH_BASE"] = f"gcs/SpiralCertaintyBin"
training_config["REMOTE_GCP_PATH_BASE"] = f"gs://tomasmuzas-phd-datasets/SpiralCertaintyBin"
training_config["DATASET_PATH"] = f"data/TFDataset/{training_config['FOLDS']}_folds"

models = [
    # {'name': 'cavanagh', 'func': Cavanagh, 'starting_fold': 1},
    {'name': 'dieleman', 'func': Dieleman, 'starting_fold': 1},
    # {'name': 'resnet50', 'func': create_ResNet50, 'starting_fold': 1},
    # {'name': 'newsimple', 'func': SimpleModel, 'starting_fold': 1}
]

perform_training(models, training_config)